## scrape data on wiki pages

In [1]:
import pandas as pd 
import requests
!pip install bs4
!pip install lxml
from bs4 import BeautifulSoup

 

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 

soup = BeautifulSoup(req.content,'lxml')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


/home/zhilin/code/private/coursera-data-sci-capstone/venv-py37/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
df

## make lat/long-itude columns using csv


In [2]:
import os


data_path = "data"
file_name = "Geospatial_Coordinates.csv"
csv_file = os.path.join(os.getcwd(), data_path, file_name)

# read la/long-titude csv
df_lalong = pd.read_csv(csv_file, names=["PostalCode", "Latitude", "Longitude"])


# join table according to colomn: Postal Code
df_joined = df.join(df_lalong.set_index('PostalCode'), on='PostalCode')
# print(df_joined["Latitude"].max())
# print(df_joined["Latitude"].min())
# print(df_joined["Longitude"].max())
# print(df_joined["Longitude"].min())
df_joined

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Queen's Park,Ontario Provincial Government,43.6623015,-79.3894938
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6536536,-79.5069436
99,M4Y,Downtown Toronto,Church and Wellesley,43.6658599,-79.3831599
100,M7Y,East Toronto Business,Enclave of M4L,43.6627439,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6362579,-79.4985091


## clustering borough according to neighborhoods
### if boroughs share neighborhoods, they should be clustered into the same group

In [3]:
# covert categorical neighborhoods data into numericals
from sklearn.preprocessing import MultiLabelBinarizer

df_ori_joined = df_joined[:]
df_joined['Neighborhood'] = df_joined['Neighborhood'].str.split(', ')
mlb = MultiLabelBinarizer()
df_onehot = df_joined.join(pd.DataFrame(mlb.fit_transform(df_joined.pop('Neighborhood')),
                          columns=mlb.classes_,
                          index=df_joined.index))
df_onehot

,PostalCode,Borough,Latitude,Longitude,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,...,Willowdale West,Wilson Heights,Woburn,Woodbine Downs,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
0,M3A,North York,43.7532586,-79.3296565,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4A,North York,43.7258823,-79.3155716,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,43.6542599,-79.3606359,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M6A,North York,43.718518,-79.4647633,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M7A,Queen's Park,43.6623015,-79.3894938,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,43.6536536,-79.5069436,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,M4Y,Downtown Toronto,43.6658599,-79.3831599,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,M7Y,East Toronto Business,43.6627439,-79.321558,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,M8Y,Etobicoke,43.6362579,-79.4985091,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# group by borough and calculate the mean of neighborhoods
df_grouped = df_onehot.groupby('Borough').mean().reset_index()
df_grouped

,Borough,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,Beaumond Heights,...,Willowdale West,Wilson Heights,Woburn,Woodbine Downs,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.111111
1,Downtown Toronto,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,Downtown Toronto Stn A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,East Toronto Business,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
5,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.25,0.25,0.000000,0.000000,0.000000,0.000000
6,East York/East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
7,Etobicoke,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,Etobicoke Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
9,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000


In [5]:
# clustering

!pip install sklearn
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5
df_grouped_clustering = df_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


array([0, 0, 0, 0, 3, 0, 0, 0, 1, 2], dtype=int32)

In [6]:
# add clustering labels
df_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
df_grouped

,Cluster Labels,Borough,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,...,Willowdale West,Wilson Heights,Woburn,Woodbine Downs,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
0,0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.111111
1,0,Downtown Toronto,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,0,Downtown Toronto Stn A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
3,0,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,3,East Toronto Business,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
5,0,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.25,0.25,0.000000,0.000000,0.000000,0.000000
6,0,East York/East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
7,0,Etobicoke,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,1,Etobicoke Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000
9,2,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000


In [7]:
# prepare data for drawing a map

df_map = df_grouped[["Borough", "Cluster Labels"]].join(df_ori_joined.set_index('Borough'), on='Borough').reset_index()

In [18]:
# convert lat/long-itude str data into float so that it can be rendered on map

df_map['Latitude'] = df_map['Latitude'].astype(float)
df_map['Longitude'] = df_map['Longitude'].astype(float)

In [20]:
# draw geo map

import folium
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors


address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_map['Latitude'], df_map['Longitude'], df_map['PostalCode'], df_map['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
